# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json as json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# data files
data_path = '../output_data/cities.csv'

# Read the data and filter results
data = pd.read_csv(data_path)
cities_df = pd.DataFrame(data)
cities_df.head()
# Store latitude and longitude in locations
locations = cities_df[["lat", "lon"]]
# Fill humidity values
humidity = cities_df["humid"]
cities_df.head()

,country,city,lat,lon,temp,max_temp,humid,cover,wind
0,US,{'albany'},42.6001,-73.9662,83.77,87.75,76,35,4.41
1,ID,{'manggar'},-2.8833,108.2667,81.90,81.90,75,41,13.42
2,TZ,{'kilindoni'},-7.9139,39.6668,75.83,75.83,69,0,14.85
3,ZA,{'east london'},-33.0153,27.9116,61.11,61.11,68,9,17.49
4,AU,{'busselton'},-33.6500,115.3333,47.05,47.05,78,35,5.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Make a new DataFrame to drop all rejected cities
nice_df = cities_df.loc[(cities_df["temp"] >= 78) &
                       (cities_df["temp"] <= 85) &
                       (cities_df["wind"] < 9) &
                       (cities_df["cover"] < 5) &
                       (cities_df["humid"] < 80 ) &
                       (cities_df["max_temp"] < 90)].copy()
nice_df = pd.DataFrame(nice_df)                       
nice_df.reset_index(inplace=True)
#len(nice_df)
nice_df.head()

,index,country,city,lat,lon,temp,max_temp,humid,cover,wind
0,24,AZ,{'mincivan'},39.0294,46.7185,78.55,78.55,39,0,0.22
1,25,US,{'saint anthony'},45.0205,-93.2180,82.51,84.13,31,1,4.00
2,162,CY,{'pafos'},34.7667,32.4167,79.70,82.18,36,0,3.44
3,216,ES,{'el prat de llobregat'},41.3278,2.0947,80.24,85.96,53,0,5.75
4,241,ES,{'san javier'},37.8063,-0.8374,80.49,84.24,69,0,1.14


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# geocoordinates
coordinates = nice_df.loc[:,["lat","lon"]]
coordinates_df = pd.DataFrame(coordinates)
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"
# hotel list
hotels = []

for i, row in coordinates_df.iterrows():
    target_coordinates = f"{row['lat']},{row['lon']}"
    # set up parms dictionary
    parms = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using parms dictionary
    try:
        response = requests.get(base_url, params=parms)
        # convert response to json
        response = response.json()
    except KeyError:
        #high level key failure
        print(f"No data returned for coordinates: {target_coordinates}")         
    except:
        #some rando failure
        print(f"No data returned for coordinates: {target_coordinates}")
    else:
        # Print the json (pretty printed)
        #print(json.dumps(response['results'], indent=4, sort_keys=True))
        try:
            hotels.append(response['results'][0]['name'])
            print(response["results"][0]["name"])
        except:
            hotels.append('none')
            print(f"No Hotel name returned for coordinates: {target_coordinates}")

# create dataframe from hotels and cities 
nice_df['hotel'] = hotels
nice_df.head()

No Hotel name returned for coordinates: 39.0294,46.7185
Hewing Hotel
Elysium Hotel
Hotel Barcelona Aeropuerto Affiliated By Meliá
Hotel Traiña
Olea Villas
Riviera 5 Star: Hotels in Beirut, Hotels In Lebanon, Beirut Hotels, Luxury Hotels Beirut
Hotel Durval
Hotel Girassol Plaza
No Hotel name returned for coordinates: 32.5,62.5
فندق مزدة الكبير
The PINES Country Inn
هتل رستوران سنتی میبد
فندق الفائز


,index,country,city,lat,lon,temp,max_temp,humid,cover,wind,hotel
0,24,AZ,{'mincivan'},39.0294,46.7185,78.55,78.55,39,0,0.22,none
1,25,US,{'saint anthony'},45.0205,-93.2180,82.51,84.13,31,1,4.00,Hewing Hotel
2,162,CY,{'pafos'},34.7667,32.4167,79.70,82.18,36,0,3.44,Elysium Hotel
3,216,ES,{'el prat de llobregat'},41.3278,2.0947,80.24,85.96,53,0,5.75,Hotel Barcelona Aeropuerto Affiliated By Meliá
4,241,ES,{'san javier'},37.8063,-0.8374,80.49,84.24,69,0,1.14,Hotel Traiña


In [6]:
drop_index = nice_df[nice_df['hotel'] == 'none'].index
nice_df.drop(drop_index, inplace=True)
nice_df.rename(columns={"hotel": "Hotel Name", "lat": "Lat", "lon": "Lng", "city": "City", "country": "Country"}, inplace=True)
hotel_df = nice_df.copy()
hotel_df.head()

,index,Country,City,Lat,Lng,temp,max_temp,humid,cover,wind,Hotel Name
1,25,US,{'saint anthony'},45.0205,-93.2180,82.51,84.13,31,1,4.00,Hewing Hotel
2,162,CY,{'pafos'},34.7667,32.4167,79.70,82.18,36,0,3.44,Elysium Hotel
3,216,ES,{'el prat de llobregat'},41.3278,2.0947,80.24,85.96,53,0,5.75,Hotel Barcelona Aeropuerto Affiliated By Meliá
4,241,ES,{'san javier'},37.8063,-0.8374,80.49,84.24,69,0,1.14,Hotel Traiña
5,298,GR,{'mouzouras'},35.5333,24.1500,80.60,80.60,74,0,5.75,Olea Villas


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_locations)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))